In [1]:
!pip install -q langchain
!pip install -q openai
!pip install -q wikipedia
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.8/304.8 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 747.5 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.1 MB/s eta 0:00:00


In [13]:
from langchain.indexes import GraphIndexCreator
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.chains import GraphQAChain

import os 
import json
import wikipedia
import networkx as nx
from langchain.graphs.networkx_graph import NetworkxEntityGraph, KnowledgeTriple
from pprint import pprint

In [10]:
os.environ["OPENAI_API_KEY"] = ""

# Step 1: Build your knowledge bases

In [11]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

def index_topic(topic="Taylor Alison Swift"):  
  indexed_kg = NetworkxEntityGraph()
  first_entity_graph = index_creator.from_text(wikipedia.summary(topic))
  triples = first_entity_graph.get_triples()
  neighbor_names = [triple[1] for triple in triples]
  for triplet in triples:
    indexed_kg.add_triple(KnowledgeTriple(triplet[0], triplet[2], triplet[1]))
  for neighbor in neighbor_names:
    try: 
      context = wikipedia.summary(neighbor)
      graph = index_creator.from_text(context)
      for triplet in graph.get_triples():
        indexed_kg.add_triple(KnowledgeTriple(triplet[0], triplet[2], triplet[1]))
    except:
      pass
  return indexed_kg

def print_knowledge(graph):
  print(len(graph.get_triples()))
  pprint(graph.get_triples())

In [17]:
# indexed_kg = index_topic("Taylor Alison Swift")
for k in indexed_kg.get_triples():
  print(k)

('Taylor Alison Swift', 'an American singer-songwriter', 'is')
('Taylor Alison Swift', 'Nashville', 'moved to')
('Taylor Alison Swift', 'Sony/ATV Music Publishing', 'signed a songwriting deal with')
('Taylor Alison Swift', 'Big Machine Records', 'signed a recording contract with')
('Taylor Alison Swift', '2006', 'released her self-titled debut album')
('Taylor Alison Swift', 'Fearless and Speak Now', 'released her next albums')
('Taylor Alison Swift', '2012', 'released her album Red')
('Taylor Alison Swift', '2014', 'released her album 1989')
('Taylor Alison Swift', '2017', 'released her album Reputation')
('Taylor Alison Swift', 'Republic Records', 'signed with')
('Taylor Alison Swift', '2019', 'released her album Lover')
('Taylor Alison Swift', '2020', 'released her album Evermore')
('Taylor Alison Swift', '2021', "released her album Red (Taylor's Version)")
('Nashville', 'U.S. state of Tennessee', 'is the capital city of')
('Nashville', 'Davidson County', 'is the seat of')
('Nashvil

# Step 2: Now given a complex question, generate a KG presentation of it to inform what hops do we need to do on the indexed knowledge.

In [301]:
question = "What album did Taylor Alison Swift release that is nominated for Best Country award?"

question: What album did Taylor Alison Swift release that is nominated for Best Country award?


## Q: What album did Taylor Alison Swift release that is nominated for Best Country award?

Questions are divided into 2 KG queries: [('Taylor Alison Swift', 'album', 'released'), ('album', 'Best Country award', 'is nominated for')]


# Step 3: Sequantial QA:

In [313]:
from langchain import PromptTemplate

template = """
I want you to turn a knowledge graph triple that describes (subjecet, object, predicate) into a question to ask. Each question start with what.
The question should be clear and succint, and it should use contain all the verbs and nouns from the triple. It should not have information from outside of the triple

Here are some examples:

- Triple: ('Adam', 'city', 'lives in') - what city does Adam live in?
- Triple: ('city', something, 'is known for') - what is something the city is known for?
- Triple: ('song', 'album A', 'was released in') - what song was released in album A?

Triple: {triple} -

"""
prompt = PromptTemplate(
    input_variables=["triple"],
    template=template,
)

def answer_complex_question(question, prompt, knowlege_graph):
  print(f"""question: {question}""")
  graph_question = index_creator.from_text(question)
  kg_queries = graph_question.get_triples()
  print(f"Questions are divided into {len(kg_queries)} KG queries: {kg_queries}\n")
  question_asking_llm = OpenAI()
  sub_questions = []
  last_answer = ""
  qa_graph_chain = GraphQAChain.from_llm(OpenAI(), graph=knowlege_graph, verbose=True)
  for i, triplet in enumerate(kg_queries):
    sub_question = question_asking_llm(prompt.format(triple=str(triplet)))
    if last_answer != "": 
      sub_question = f"""Among {result["result"]}, {sub_question}"""
    print(f"\n\n\n Step #{i+1}: query {kg_queries[i]} turned into sub_question: {sub_question}")
    result = qa_graph_chain(sub_question)
    print( f"""Question: {result["query"]} \nAnswer:  {result["result"]}""")  
    last_answer = result["result"]
    if "I don't know" in last_answer:
      last_answer = ""
  
  print("/n Final answer: ", last_answer)

In [314]:
answer_complex_question(question, prompt, indexed_kg)

question: What album did Taylor Alison Swift release that is nominated for Best Country award?
Questions are divided into 2 KG queries: [('Taylor Alison Swift', 'album', 'released'), ('album', 'Best Country award', 'is nominated for')]




 Step #1: query ('Taylor Alison Swift', 'album', 'released') turned into sub_question: What album was released by Taylor Alison Swift?


> Entering new GraphQAChain chain...
Entities Extracted:
 Taylor Alison Swift
Full Context:
Taylor Alison Swift is an American singer-songwriter
Taylor Alison Swift is known for her genre-spanning discography
Taylor Alison Swift is known for her artistic reinventions
Taylor Alison Swift is known for her songwriting
Taylor Alison Swift moved to Nashville
Taylor Alison Swift signed a songwriting deal with Sony/ATV Music Publishing
Taylor Alison Swift signed a recording contract with Big Machine Records
Taylor Alison Swift released her debut album self-titled
Taylor Alison Swift released her next albums Fearless and Sp

In [278]:
graph_question = index_creator.from_text(question)
graph_question.get_triples()

[('Taylor Alison Swift', 'album', 'released'),
 ('album', 'Best Country award', 'is nominated for')]

In [279]:
question_asking_llm = OpenAI()

In [280]:
questions = []
for triplet in graph_question.get_triples():
  question = question_asking_llm(prompt.format(triple=str(triplet)))
  print(triplet, question)
  questions.append(question)

('Taylor Alison Swift', 'album', 'released') What album has Taylor Alison Swift released?
('album', 'Best Country award', 'is nominated for') What album is nominated for the Best Country award?


In [284]:
qa_chain = GraphQAChain.from_llm(OpenAI(), graph=indexed_kg, verbose=True)
result = qa_chain(questions[0])
print( "1st Question", result["query"], "1st answer: ", result["result"])

next_question = f"""Among {result["result"]}, {questions[1]}"""
final_result = qa_chain(next_question)
print( "2nd Question", final_result["query"], "2nd answer", final_result["result"])





> Entering new GraphQAChain chain...
Entities Extracted:
 Taylor Alison Swift
Full Context:
Taylor Alison Swift is an American singer-songwriter
Taylor Alison Swift is known for her genre-spanning discography
Taylor Alison Swift is known for her artistic reinventions
Taylor Alison Swift is known for her songwriting
Taylor Alison Swift moved to Nashville
Taylor Alison Swift signed a songwriting deal with Sony/ATV Music Publishing
Taylor Alison Swift signed a recording contract with Big Machine Records
Taylor Alison Swift released her debut album self-titled
Taylor Alison Swift released her next albums Fearless and Speak Now
Taylor Alison Swift released her single "Love Story"
Taylor Alison Swift released her single "You Belong with Me"
Taylor Alison Swift released her album Red
Taylor Alison Swift released her single "We Are Never Ever Getting Back Together"
Taylor Alison Swift released her album 1989
Taylor Alison Swift released her singles "S

> Finished chain.
1st Question What alb